## Zeitreihenanalyse Covid-19 Europa

Datenquelle: Bereinigte Daten der Johns Hopkins Unviversity

Analyse der 7-Tage-Inzidenzen je 100.000 Einwohner

In [40]:
import pandas as pd 
import folium
from selenium import webdriver
import time
import os

Hier holen wir uns die bereinigten Daten der John-Hopskins-University.

In [41]:
covid_df = pd.read_csv('https://raw.githubusercontent.com/Lucas-Czarnecki/COVID-19-CLEANED-JHUCSSE/master/COVID-19_CLEAN/csse_covid_19_time_series_cleaned/time_series_covid19_cases_tidy.csv',parse_dates=['Date'])

covid_df = covid_df.drop(columns=['Province_State', 'Latitude', 'Longitude'])

covid_df.rename(columns=dict(Country_Region = 'land', Confirmed = 'infiziert', Deaths = 'tot', Recovered = 'erholt', Date = 'datum'),inplace=True)

Jetzt holen wir uns die Ländercodes (ISO 3-stellig) aller Länder

In [42]:
country_codes = pd.read_csv('https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv')
country_codes.rename(columns={'ISO3166-1-Alpha-3':'iso3', 'official_name_en':'iso_land'},inplace=True)
country_codes = country_codes[country_codes['Region Name'] == 'Europe']
country_codes = country_codes[['iso3','iso_land']]

Leider muss man etliche Ländernamen anpassen, damit die Tabellen synchron sind.

In [43]:
convert = {'United Kingdom of Great Britain and Northern Ireland':'United Kingdom', 'Republic of Moldova':'Moldova', 'Russian Federation':'Russia'}
old = convert.values()
new = convert.keys()
covid_df.land = covid_df.land.replace(old,new)


In [44]:
not_in_country_codes = covid_df[~covid_df.land.isin(country_codes.iso_land)]
not_in_country_codes.land.unique()

array(['Canada', 'China', 'Australia', 'Afghanistan', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Botswana', 'Brazil', 'Brunei',
       'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia',
       'Cameroon', 'Central African Republic', 'Chad', 'Chile',
       'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', "Cote d'Ivoire", 'Cuba', 'Cyprus',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Eswatini', 'Ethiopia', 'Fiji', 'Gabon', 'Gambia', 'Georgia',
       'Ghana', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau',
       'Guyana', 'Haiti', 'Honduras', 'India', 'Indonesia', 'Iran',
       'Iraq', 'Israel', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan',
       'Kenya', 'South Korea', 'Kosovo', 

Hier findet das Zusammenführen der ISO-Codes mit der COVID-Tabelle statt

In [45]:
covid_df = pd.merge(covid_df, country_codes, left_on=['land'], right_on=['iso_land'])


In [46]:
pop_df = pd.read_csv('https://raw.githubusercontent.com/datasets/population/master/data/population.csv')
pop_df = pop_df[pop_df.Year == 2018]

Über die die ISO-Codes
 erfolgt eine Zusammenführung der Covid-Tabelle mit den Bevölerkungszahlen. Nicht sehr speichereffizient, da jede Zeile des Landes für jeden Tag jetzt auch die Bevölkerungszahl enthält, aber dafür einfach in der weiteren Berechnung von Infizierten/100.000 Einwohner

In [47]:
covid_df = pd.merge(covid_df, pop_df, left_on=['iso3'], right_on=['Country Code'])
covid_df = covid_df.drop(columns=['iso_land', 'Country Name', 'Country Code', 'Year'])

Jetzt ermitteln wir die Neu Infizierten kumuliert über die letzten 7 Tage und umgerechnet auf 100.000 Einwohner

In [48]:
# Berechnung von weiteren Daten auf der Basis der vorhandenen Informationen
covid_df['neuInfiziert'] = covid_df.infiziert.diff(7) / covid_df.Value * 100_000
covid_df['toteNorm'] = covid_df.tot / covid_df.Value * 100_000
covid_df.head()

,land,datum,infiziert,tot,erholt,iso3,Value,neuInfiziert,toteNorm
0,United Kingdom of Great Britain and Northern I...,2020-01-22,0,0,0.0,GBR,66460344,NaN,0.0
1,United Kingdom of Great Britain and Northern I...,2020-01-23,0,0,0.0,GBR,66460344,NaN,0.0
2,United Kingdom of Great Britain and Northern I...,2020-01-24,0,0,0.0,GBR,66460344,NaN,0.0
3,United Kingdom of Great Britain and Northern I...,2020-01-25,0,0,0.0,GBR,66460344,NaN,0.0
4,United Kingdom of Great Britain and Northern I...,2020-01-26,0,0,0.0,GBR,66460344,NaN,0.0


Hier holen wir uns die GEO-JSON-Daten der Umrisse der einzelnen Länder. Dann ermitteln wir alle ISO-Ländercodes von Ländern aus der GEO-JSON-Tabelle und schmeißen danach alle Länder aus der COVID-Datenanalyse, die nicht in der GEO-JSON sind.

Wenn wir keinen Umriss haben, brauchen wir uns um die Daten dieses Landes nicht zu kümmern.

In [49]:
geo_iso = set()
geo_json = pd.read_json('https://raw.githubusercontent.com/austinkeeley/Countries-GeoJSON/master/countries.json')
for row in geo_json.features:
    geo_iso.add(row['properties']['iso_a3'])
not_in_geo = covid_df[~covid_df['iso3'].isin(geo_iso)]
not_in_geo.iso3.unique()
covid_df = covid_df[~covid_df.iso3.isin(not_in_geo.iso3.unique())]   

Hier fliegen jetzt alle Länder raus, deren Einwohnerzahl unter 3 Mio. liegen. Dadurch wird verhindert, dass irgenwelche kleinen Länder mit Ihren Infektionszahlen (schönes Beispiel "Lichtenstein") den Schnitt weltweit dominieren.

In [50]:
covid_df = covid_df[covid_df.Value > 3_000_000]   

In [38]:
counter = start_counter = 233
geo_data_raw = r'https://raw.githubusercontent.com/austinkeeley/Countries-GeoJSON/master/countries.json'
for date in covid_df.datum.unique()[start_counter:]:
    df = covid_df[covid_df.datum == date]
    d = pd.to_datetime(str(date))
    d = d.strftime('%d.%m.%Y')
    title_html = f'<h3 align="center" style="font-size:20px"><b>7-Tage-Inzidenzen je 100.000 Einwohner vom {d}</b></h3>'
    m = folium.Map(width=1920,height=1200,location = [53.551086, 9.993682], zoom_start=5)
    m.choropleth(
        geo_data= geo_data_raw,
        data= df,
        columns=['iso3', 'neuInfiziert'],
        key_on = 'feature.properties.iso_a3',
        fill_color = 'YlOrRd',
        nan_fill_color = 'white',
        fill_opacity = 0.7,
        line_opacity = 0.2,
        legend_name = 'Summe der Neuinfektionen der letzten 7 Tage je 100.000 Einwohner',
        highlight = True
    )
    m.get_root().html.add_child(folium.Element(title_html))
    m.save(f'Teil_35_Maps\Europe{counter:03d}.html')
    counter += 1

In [39]:
delay=5
browser = webdriver.Chrome('d:\programme\Selenium_web_driver\chromedriver.exe')
for file in os.listdir('Teil_35_Maps'):
    if not file.endswith('.html') or not file.startswith('Europe') or int(file[6:9]) < start_counter: continue
    browser.get(f'file:///D:/daten/Sprachen/Python/A-beautiful-code-in-Python/Teil_35_Maps/{file}')
    #Give the map tiles some time to load
    time.sleep(5)
    #Grab the screenshot
    browser.save_screenshot(f'D:/daten/Sprachen/Python/A-beautiful-code-in-Python/Teil_35_Maps/{file}.png')
#Close the browser
browser.quit()